In [45]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D
from keras import backend as K

In [33]:
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_sample = 1000
nb_validation_samples = 100
epochs_number = 20
batch_size = 20

In [34]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [35]:
train_datagen = ImageDataGenerator(
    rescale = 1. / 255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

In [36]:
test_datagen = ImageDataGenerator(rescale = 1. / 255)

In [37]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary')

Found 8005 images belonging to 2 classes.


In [38]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary')

Found 2023 images belonging to 2 classes.


In [39]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_11 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_12 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 34, 34, 64)       

In [40]:
model.compile(loss = 'binary_crossentropy',
             optimizer = 'rmsprop',
             metrics = ['accuracy'])

In [51]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs={}):
        if(logs.get('val_accuracy')>0.99):
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training=True

In [52]:
made_callback = myCallback()

In [53]:
model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_sample,
    epochs = epochs_number,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples,
    callbacks=[made_callback]
    )

Epoch 1/20
1000/1000 [==============================] - 258s 258ms/step - loss: 0.4815 - accuracy: 0.7796 - val_loss: 0.4593 - val_accuracy: 0.7605
Epoch 2/20
1000/1000 [==============================] - 245s 245ms/step - loss: 0.4555 - accuracy: 0.7993 - val_loss: 0.3677 - val_accuracy: 0.8195
Epoch 3/20
1000/1000 [==============================] - 241s 241ms/step - loss: 0.4405 - accuracy: 0.8063 - val_loss: 0.2829 - val_accuracy: 0.8139
Epoch 4/20
1000/1000 [==============================] - 245s 245ms/step - loss: 0.4402 - accuracy: 0.8078 - val_loss: 0.5968 - val_accuracy: 0.7867
Epoch 5/20
1000/1000 [==============================] - 257s 257ms/step - loss: 0.4279 - accuracy: 0.8152 - val_loss: 0.3388 - val_accuracy: 0.8210
Epoch 6/20
1000/1000 [==============================] - 240s 240ms/step - loss: 0.4177 - accuracy: 0.8172 - val_loss: 0.8778 - val_accuracy: 0.8371
Epoch 7/20
1000/1000 [==============================] - 244s 244ms/step - loss: 0.4215 - accuracy: 0.8221 - val_

In [57]:
img_pred = image.load_img('data/test/dogs/dog.4041.jpg', target_size = (150, 150))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred, axis = 0)

In [58]:
result = model.predict(img_pred)
print (result)
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
    
print (prediction)

[[1.]]
dog
